# Import libraries

In [1]:
import pandas as pd
import numpy as np
import string as pystring
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras import backend as K

# Load Dataset

In [2]:
!gdown --id 1USrci90pyG2E76_VFH1igIYj2i8BvlmE

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1USrci90pyG2E76_VFH1igIYj2i8BvlmE
To: /content/truyen_kieu.txt
100% 140k/140k [00:00<00:00, 96.3MB/s]


In [3]:
with open('truyen_kieu.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Preprocessing

In [4]:
PUNCT_TO_REMOVE = pystring.punctuation + pystring.digits + "\n"
def clean_text (text):
  """custom function to removal: punctuations and digits"""
  text = text.translate(str.maketrans(' ',' ', PUNCT_TO_REMOVE))
  text = text.lower()
  return text
clean_text(lines[0])

'trăm năm trong cõi người ta'

In [5]:
corpus = [clean_text(line) for line in lines]
print(corpus[0])

trăm năm trong cõi người ta


# Build Data

In [6]:
def get_centers_and_contexts(corpus, max_window_size=2):
  centers, contexts = [], []
  for line in corpus:

    line = line.split()

    if len(line) <= 2*max_window_size:
      continue
    for i in range(max_window_size, len(line) - max_window_size):
      centers.append(line[i])
      idxs = list(range(i-max_window_size, i+max_window_size+1))
      idxs.remove(i)
      contexts.append(" ".join([line[idx] for idx in idxs]))
  return centers, contexts

In [7]:
centers, contexts = get_centers_and_contexts(corpus)
len(centers), len(contexts)

(9778, 9778)

In [8]:
centers[:2], contexts[:2]

(['trong', 'cõi'], ['trăm năm cõi người', 'năm trong người ta'])

# Representation

In [9]:
max_length = 4
embedding_size = 200

In [10]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)

In [11]:
vocab_size = len(tokenizer.index_word) + 1

In [12]:
train_seq = tokenizer.texts_to_sequences(contexts)
train_seq_pad = pad_sequences(train_seq, maxlen=max_length, truncating='post', padding="post")

In [13]:
train_labels = [to_categorical(tokenizer.word_index[label], len(tokenizer.word_index) + 1) for label in centers]

In [14]:
train_labels = np.array(train_labels)

# CBOW Model

In [15]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_length))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 200)            482400    
                                                                 
 lambda (Lambda)             (None, 200)               0         
                                                                 
 dense (Dense)               (None, 2412)              484812    
                                                                 
Total params: 967212 (3.69 MB)
Trainable params: 967212 (3.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
cbow.fit(train_seq_pad, train_labels, epochs=30, verbose=1)

Epoch 1/30
306/306 [==============================] - 8s 19ms/step - loss: 7.4536 - acc: 0.0183
Epoch 2/30
306/306 [==============================] - 2s 7ms/step - loss: 6.7264 - acc: 0.0205
Epoch 3/30
306/306 [==============================] - 2s 6ms/step - loss: 6.5675 - acc: 0.0240
Epoch 4/30
306/306 [==============================] - 2s 5ms/step - loss: 6.3946 - acc: 0.0353
Epoch 5/30
306/306 [==============================] - 2s 5ms/step - loss: 6.1507 - acc: 0.0532
Epoch 6/30
306/306 [==============================] - 2s 8ms/step - loss: 5.8244 - acc: 0.0782
Epoch 7/30
306/306 [==============================] - 2s 5ms/step - loss: 5.4259 - acc: 0.1132
Epoch 8/30
306/306 [==============================] - 2s 5ms/step - loss: 4.9736 - acc: 0.1545
Epoch 9/30
306/306 [==============================] - 1s 5ms/step - loss: 4.4889 - acc: 0.2064
Epoch 10/30
306/306 [==============================] - 1s 5ms/step - loss: 3.9984 - acc: 0.2716
Epoch 11/30
306/306 [===========================

# Predict

In [17]:
sample_text = 'trăm năm cõi người'
sample_seq = tokenizer.texts_to_sequences([sample_text])
sample_seq_pad = pad_sequences(sample_seq, maxlen=max_length, truncating='post', padding="post")
cbow.predict(sample_seq_pad)

1/1 [==============================] - 0s 181ms/step


array([[6.1588022e-12, 6.0259302e-12, 1.2421175e-03, ..., 4.2154522e-08,
        6.2985797e-12, 6.2523782e-08]], dtype=float32)

In [18]:
tokenizer.index_word[np.argmax(cbow.predict(sample_seq_pad))]

1/1 [==============================] - 0s 47ms/step


'trong'